In [18]:
import pandas as pd

train_df = pd.read_csv('C:/Users/breath03/Desktop/Smart_Closet/labeled_fashion_data.csv')
# print(train_df.head())
print(train_df.shape)

C:\Users\breath03\AppData\Local\Temp\ipykernel_18652\3686205631.py:3: DtypeWarning: Columns (25,62,89,229,230,231,232,233,234,235,236,237,340,342,343,344,345,347,348,349,508,707,708) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('C:/Users/breath03/Desktop/Smart_Closet/labeled_fashion_data.csv')


(21240, 1189)


In [ ]:
# 폴리곤에 대한 값을 계산해서 새로운 피처로 추가하는 코드

import pandas as pd
from shapely.geometry import Polygon

# --- 피처 엔지니어링을 적용할 의류 카테고리 목록 ---
clothing_parts = ["아우터", "하의", "원피스", "상의"]

# 삭제할 원본 폴리곤 컬럼들을 저장할 리스트
all_poly_cols_to_drop = []

# 각 의류 카테고리별로 반복 작업 수행
for part in clothing_parts:
    print(f"--- '{part}' 카테고리 폴리곤 피처 생성 시작 ---")

    # 1. 현재 카테고리에 해당하는 폴리곤 좌표 컬럼 이름들을 동적으로 찾기
    # 예: part가 '상의'이면 '상의_폴리곤_X좌표'가 포함된 컬럼을 찾음
    poly_x_cols = sorted([col for col in train_df.columns if f'{part}_폴리곤_X좌표' in col])
    poly_y_cols = sorted([col for col in train_df.columns if f'{part}_폴리곤_Y좌표' in col])
    
    # 해당 카테고리의 폴리곤 좌표 데이터가 없으면 다음 카테고리로 넘어감
    if not poly_x_cols:
        print(f"'{part}' 카테고리에 대한 폴리곤 좌표 정보가 없어 건너뜁니다.")
        continue

    # 삭제 리스트에 현재 좌표 컬럼들 추가
    all_poly_cols_to_drop.extend(poly_x_cols)
    all_poly_cols_to_drop.extend(poly_y_cols)
    
    # 각 행(이미지)마다 (X, Y) 좌표 쌍 리스트를 만드는 함수 (재사용)
    def create_polygon_points(row, x_cols, y_cols):
        points = []
        for x_col, y_col in zip(x_cols, y_cols):
            if pd.notna(row[x_col]) and pd.notna(row[y_col]):
                points.append((row[x_col], row[y_col]))
        if len(points) >= 3:
            return Polygon(points)
        return None

    # 임시 'polygon' 컬럼 생성
    temp_polygon_col = f'{part}_polygon'
    train_df[temp_polygon_col] = train_df.apply(lambda row: create_polygon_points(row, poly_x_cols, poly_y_cols), axis=1)

    # 2. 새로운 피처(Feature)를 동적 컬럼명으로 계산 및 추가
    train_df[f'{part}_폴리곤_면적'] = train_df[temp_polygon_col].apply(lambda p: p.area if p else 0)

    def get_bbox_features(p):
        if p:
            minx, miny, maxx, maxy = p.bounds
            width = maxx - minx
            height = maxy - miny
            aspect_ratio = width / height if height > 0 else 0
            return width, height, aspect_ratio
        return 0, 0, 0

    bbox_features = train_df[temp_polygon_col].apply(get_bbox_features)
    train_df[[f'{part}_폴리곤_너비', f'{part}_폴리곤_높이', f'{part}_폴리곤_가로세로비']] = pd.DataFrame(bbox_features.tolist(), index=train_df.index)

    train_df[f'{part}_폴리곤_중심X'] = train_df[temp_polygon_col].apply(lambda p: p.centroid.x if p else 0)
    train_df[f'{part}_폴리곤_중심Y'] = train_df[temp_polygon_col].apply(lambda p: p.centroid.y if p else 0)
    
    # 임시 polygon 컬럼은 삭제 리스트에 추가
    all_poly_cols_to_drop.append(temp_polygon_col)
    print(f"'{part}' 카테고리 처리 완료.")


# 3. 모든 원본 폴리곤 좌표 컬럼 및 임시 컬럼들을 한 번에 제거
print("\n--- 불필요한 원본 폴리곤 좌표 컬럼 전체 삭제 중 ---")
train_df.drop(columns=all_poly_cols_to_drop, inplace=True, errors='ignore')

# 결과 확인
print("\n최종 DataFrame Info:")
train_df.info()

# 생성된 피처 중 일부를 확인
print("\n새롭게 생성된 피처 확인 (상위 5개):")
new_feature_cols = [col for col in train_df.columns if '_폴리곤_' in col]
print(train_df[['이미지 식별자'] + new_feature_cols].head())

# 파일로 저장
train_df.to_csv('labeled_fashion_data(polygon_features).csv', index=False, encoding='utf-8-sig')
print("\n'labeled_fashion_data(polygon_features).csv' 파일로 저장이 완료되었습니다.")

--- '아우터' 카테고리 폴리곤 피처 생성 시작 ---
'아우터' 카테고리 처리 완료.
--- '하의' 카테고리 폴리곤 피처 생성 시작 ---
'하의' 카테고리 처리 완료.
--- '원피스' 카테고리 폴리곤 피처 생성 시작 ---
'원피스' 카테고리 처리 완료.
--- '상의' 카테고리 폴리곤 피처 생성 시작 ---
'상의' 카테고리 처리 완료.

--- 불필요한 원본 폴리곤 좌표 컬럼 전체 삭제 중 ---

최종 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21240 entries, 0 to 21239
Data columns (total 89 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   이미지 식별자        21240 non-null  int64  
 1   이미지 높이         21240 non-null  int64  
 2   이미지 파일명        21240 non-null  object 
 3   이미지 너비         21240 non-null  int64  
 4   파일 생성일자        21240 non-null  object 
 5   카테고리           21240 non-null  object 
 6   스타일_스타일        21240 non-null  object 
 7   하의_렉트_X좌표      11399 non-null  float64
 8   하의_렉트_Y좌표      11399 non-null  float64
 9   하의_렉트_가로       11399 non-null  float64
 10  하의_렉트_세로       11399 non-null  float64
 11  하의_라벨_기장       11359 non-null  object 
 12  하의_라벨_색상       1168

In [ ]:
# yolo 학습에 필요한 .txt파일 생성 코드

import pandas as pd
import os

def create_yolo_dataframe(csv_path):
    """
    원본 CSV 파일에서 YOLO 학습에 필요한 컬럼만 추출하여
    새로운 데이터프레임(train_yolo_df)을 생성합니다.
    """
    print(f"'{csv_path}' 파일을 읽는 중입니다...")
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"오류: '{csv_path}' 파일을 찾을 수 없습니다.")
        return None

    # YOLO 학습에 필요한 컬럼만 담을 리스트 초기화
    yolo_data_list = []
    item_types = ['상의', '하의', '아우터', '원피스']

    print("YOLO 학습에 필요한 데이터 추출을 시작합니다...")
    # 원본 데이터프레임 순회
    for index, row in df.iterrows():
        image_identifier = str(row['이미지 식별자'])
        image_width = row['이미지 너비']
        image_height = row['이미지 높이']

        for item_type in item_types:
            x_col, y_col, w_col, h_col = f'{item_type}_렉트_X좌표', f'{item_type}_렉트_Y좌표', f'{item_type}_렉트_가로', f'{item_type}_렉트_세로'

            # 해당 아이템의 좌표 정보가 있는지 확인 (0보다 큰 경우)
            if x_col in row and row.get(x_col, 0) > 0:
                yolo_data_list.append({
                    'image_identifier': image_identifier,
                    'image_width': image_width,
                    'image_height': image_height,
                    'class': item_type,
                    'x': row[x_col],
                    'y': row[y_col],
                    'width': row[w_col],
                    'height': row[h_col]
                })

    # 새로운 데이터프레임 생성
    train_yolo_df = pd.DataFrame(yolo_data_list)
    print("추출 완료! 'train_yolo_df' 데이터프레임을 생성했습니다.")
    return train_yolo_df


def create_yolo_label_files(yolo_df, output_dir='labels'):
    """
    train_yolo_df를 입력받아 YOLO 형식의 .txt 라벨 파일을 생성합니다.
    """
    if yolo_df is None or yolo_df.empty:
        print("데이터프레임이 비어있어 라벨 파일을 생성할 수 없습니다.")
        return

    # 라벨을 저장할 폴더 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 클래스 이름을 숫자 ID로 매핑 (0부터 시작)
    class_to_id = {name: i for i, name in enumerate(yolo_df['class'].unique())}
    print("\n클래스 매핑 정보:")
    print(class_to_id)

    print(f"\n'{output_dir}' 폴더에 YOLO 라벨 파일 생성을 시작합니다...")
    
    # 이미지 식별자별로 그룹화하여 처리
    for image_id, group in yolo_df.groupby('image_identifier'):
        label_path = os.path.join(output_dir, f"{image_id}.txt")
        
        with open(label_path, 'w') as f:
            for _, row in group.iterrows():
                # 이미지 크기
                img_w, img_h = row['image_width'], row['image_height']
                
                # Bounding Box 좌표 (x, y, 너비, 높이)
                x, y, w, h = row['x'], row['y'], row['width'], row['height']

                # YOLO 형식으로 변환 (중심 x, 중심 y, 너비, 높이) 및 정규화
                center_x = (x + w / 2) / img_w
                center_y = (y + h / 2) / img_h
                norm_w = w / img_w
                norm_h = h / img_h
                
                # 클래스 ID 가져오기
                class_id = class_to_id[row['class']]

                # 파일에 쓰기
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}\n")
    
    print(f"라벨 파일 생성이 완료되었습니다. 총 {len(yolo_df.groupby('image_identifier'))}개의 파일이 생성되었습니다.")
    # YOLO 학습 설정에 필요한 YAML 파일 내용 출력
    print("\n--- YOLO 학습용 YAML 파일에 다음 내용을 추가하세요 ---")
    print("names:")
    for name, class_id in class_to_id.items():
        print(f"  {class_id}: {name}")
    print(f"nc: {len(class_to_id)}")
    print("-------------------------------------------------")


# --- 메인 코드 실행 ---
if __name__ == '__main__':
    # 1. 원본 CSV 파일에서 필요한 데이터만 추출하여 DataFrame 생성
    csv_file = 'C:/Users/breath03/Desktop/Smart_Closet/labeled_fashion_data(polygon_features).csv'
    train_yolo_df = create_yolo_dataframe(csv_file)

    # 생성된 DataFrame 확인 (상위 5개)
    if train_yolo_df is not None:
        print("\n생성된 train_yolo_df 미리보기:")
        print(train_yolo_df.head())

        # 2. 생성된 DataFrame을 바탕으로 YOLO .txt 라벨 파일들 생성
        create_yolo_label_files(train_yolo_df, output_dir='yolo_labels')

'C:/Users/breath03/Desktop/Smart_Closet/labeled_fashion_data(polygon_features).csv' 파일을 읽는 중입니다...
YOLO 학습에 필요한 데이터 추출을 시작합니다...
추출 완료! 'train_yolo_df' 데이터프레임을 생성했습니다.

생성된 train_yolo_df 미리보기:
  image_identifier  image_width  image_height class        x         y  width  \
0           100026          800           800    상의  260.144   59.4236  295.0   
1           100026          800           800    하의  321.060  288.2160  183.0   
2           100027          800           800    상의  188.244   51.4347  453.0   
3           100027          800           800    하의  285.110  369.2150  263.0   
4           100028          800           800    상의  100.367   71.4069  420.0   

   height  
0   335.0  
1   330.0  
2   468.0  
3   427.0  
4   523.0  

클래스 매핑 정보:
{'상의': 0, '하의': 1, '원피스': 2, '아우터': 3}

'yolo_labels' 폴더에 YOLO 라벨 파일 생성을 시작합니다...
라벨 파일 생성이 완료되었습니다. 총 21039개의 파일이 생성되었습니다.

--- YOLO 학습용 YAML 파일에 다음 내용을 추가하세요 ---
names:
  0: 상의
  1: 하의
  2: 원피스
  3: 아우터
nc: 4
-------------------------

In [ ]:
# yolo모델에 학습에 쓰일 이미지 폴더 생성 코드

import os
import shutil
import pandas as pd

# --- 설정 (백지민님이 확인할 부분) ---
# 1. 원본 데이터 경로
# 이전에 'prepare_yolo_data.py'를 실행했던 폴더와 동일한 위치에서 실행해주세요.
original_image_root = 'C:/Users/breath03/Desktop/Smart_Closet/Training/원천데이터' # 원본 룩 사진들이 있는 최상위 폴더
label_folder = 'C:/Users/breath03/Desktop/Smart_Closet/yolo_labels'             # 이전에 생성한 .txt 라벨 파일 폴더

# 2. 새로 생성될 YOLO 학습용 이미지 폴더
output_image_folder = 'C:/Users/breath03/Desktop/Smart_Closet/yolo_images'
# -----------------------------------------

def create_yolo_image_folder():
    """
    라벨(.txt) 파일이 존재하는 이미지들만 원본 폴더에서 찾아
    새로운 학습용 이미지 폴더로 복사합니다.
    """
    # 결과물 저장 폴더 생성
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)

    # 1. 원본 이미지 경로 맵 생성
    print(f"'{original_image_root}'와 모든 하위 폴더에서 원본 이미지 파일을 찾는 중입니다...")
    if not os.path.isdir(original_image_root):
        print(f"오류: '{original_image_root}' 디렉토리를 찾을 수 없습니다.")
        return
        
    image_path_map = {}
    for root, _, files in os.walk(original_image_root):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                identifier = os.path.splitext(file)[0]
                image_path_map[identifier] = os.path.join(root, file)
    print(f"총 {len(image_path_map)}개의 원본 이미지 경로를 찾았습니다.")
    
    # 2. 라벨 파일 목록 가져오기
    if not os.path.isdir(label_folder):
        print(f"오류: '{label_folder}' 디렉토리를 찾을 수 없습니다. 이전 단계에서 라벨 파일을 먼저 생성해주세요.")
        return
        
    label_files = os.listdir(label_folder)
    print(f"'{label_folder}'에서 총 {len(label_files)}개의 라벨 파일을 찾았습니다.")

    # 3. 이미지 복사 시작
    copied_count = 0
    print(f"\n'{output_image_folder}' 폴더로 이미지 복사를 시작합니다...")
    for label_file in label_files:
        # 라벨 파일 이름에서 확장자를 제거하여 이미지 식별자 추출 (예: '994962')
        identifier = os.path.splitext(label_file)[0]

        # 해당 식별자를 가진 원본 이미지가 있는지 확인
        if identifier in image_path_map:
            source_path = image_path_map[identifier]
            # 복사될 파일의 전체 경로
            destination_path = os.path.join(output_image_folder, os.path.basename(source_path))
            
            # 이미지 파일 복사
            shutil.copyfile(source_path, destination_path)
            copied_count += 1
        
        if (copied_count % 1000 == 0) and (copied_count > 0):
             print(f"{copied_count}개의 이미지 복사 완료...")

    print("\n" + "="*50)
    print("🎉 YOLO 학습용 이미지 폴더 생성이 완료되었습니다! 🎉")
    print(f"🖼️  총 {copied_count}개의 이미지를 '{output_image_folder}' 폴더에 복사했습니다.")
    print("="*50)

# --- 메인 코드 실행 ---
if __name__ == '__main__':
    create_yolo_image_folder()

'C:/Users/breath03/Desktop/Smart_Closet/Training/원천데이터'와 모든 하위 폴더에서 원본 이미지 파일을 찾는 중입니다...
총 643265개의 원본 이미지 경로를 찾았습니다.
'C:/Users/breath03/Desktop/Smart_Closet/yolo_labels'에서 총 21039개의 라벨 파일을 찾았습니다.

'C:/Users/breath03/Desktop/Smart_Closet/yolo_images' 폴더로 이미지 복사를 시작합니다...
1000개의 이미지 복사 완료...
2000개의 이미지 복사 완료...
3000개의 이미지 복사 완료...
4000개의 이미지 복사 완료...
5000개의 이미지 복사 완료...
6000개의 이미지 복사 완료...
7000개의 이미지 복사 완료...
8000개의 이미지 복사 완료...
9000개의 이미지 복사 완료...
10000개의 이미지 복사 완료...
11000개의 이미지 복사 완료...
12000개의 이미지 복사 완료...
13000개의 이미지 복사 완료...
14000개의 이미지 복사 완료...
15000개의 이미지 복사 완료...
16000개의 이미지 복사 완료...
17000개의 이미지 복사 완료...
18000개의 이미지 복사 완료...
19000개의 이미지 복사 완료...
20000개의 이미지 복사 완료...
21000개의 이미지 복사 완료...

🎉 YOLO 학습용 이미지 폴더 생성이 완료되었습니다! 🎉
🖼️  총 21039개의 이미지를 'C:/Users/breath03/Desktop/Smart_Closet/yolo_images' 폴더에 복사했습니다.


In [ ]:
# yolo 학습에 쓰일 이미지/라벨 폴더를 학습/검증용으로 나누는 코드

import os
import random
import shutil

# --- 설정 ---
# 원본 폴더 경로
image_dir = 'C:/Users/breath03/Desktop/Smart_Closet/yolo_images'
label_dir = 'C:/Users/breath03/Desktop/Smart_Closet/yolo_labels'

# 새로 만들 폴더 경로
base_output_dir = 'C:/Users/breath03/Desktop/Smart_Closet/fashion_dataset'

# 검증 데이터셋 비율 (예: 0.2 = 20%)
val_split_ratio = 0.2
# -----------

def split_data():
    # 전체 이미지 파일 목록 가져오기
    all_images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    random.shuffle(all_images)

    # 학습용과 검증용으로 나누기
    split_index = int(len(all_images) * (1 - val_split_ratio))
    train_files = all_images[:split_index]
    val_files = all_images[split_index:]

    print(f"총 이미지 수: {len(all_images)}")
    print(f"학습용 데이터 수: {len(train_files)}")
    print(f"검증용 데이터 수: {len(val_files)}")

    # 폴더 생성 함수
    def create_dirs(base, set_name):
        os.makedirs(os.path.join(base, 'images', set_name), exist_ok=True)
        os.makedirs(os.path.join(base, 'labels', set_name), exist_ok=True)

    # 파일 복사 함수
    def copy_files(files, set_name):
        for filename in files:
            basename = os.path.splitext(filename)[0]
            # 이미지 복사
            shutil.copy(os.path.join(image_dir, filename), 
                        os.path.join(base_output_dir, 'images', set_name, filename))
            # 라벨 복사
            shutil.copy(os.path.join(label_dir, f"{basename}.txt"), 
                        os.path.join(base_output_dir, 'labels', set_name, f"{basename}.txt"))

    # 폴더 생성 및 파일 복사 실행
    create_dirs(base_output_dir, 'train')
    create_dirs(base_output_dir, 'val')
    copy_files(train_files, 'train')
    copy_files(val_files, 'val')

    print(f"\n'/{base_output_dir}/' 폴더에 학습/검증 데이터셋 생성을 완료했습니다.")

if __name__ == '__main__':
    split_data()

총 이미지 수: 21039
학습용 데이터 수: 16831
검증용 데이터 수: 4208

'/C:/Users/breath03/Desktop/Smart_Closet/fashion_dataset/' 폴더에 학습/검증 데이터셋 생성을 완료했습니다.


In [ ]:
# yolo 모델 실제 실행해보는 코드

from ultralytics import YOLO
from PIL import Image
import os

# --- 설정 (백지민님이 수정해야 할 부분) ---

# 1. 훈련된 나만의 모델 경로를 지정해주세요.
# 보통 'runs/detect/train/weights/best.pt' 형태입니다.
# train 뒤의 숫자는 실행할 때마다 바뀔 수 있으니, 폴더를 직접 확인해보세요.
MODEL_PATH = 'C:/Users/breath03/runs/detect/train5/weights/best.pt' 

# 2. 테스트하고 싶은 이미지 파일들의 경로를 리스트에 담아주세요.
# 스크립트와 같은 폴더에 이미지들을 두는 것이 가장 간단합니다.
IMAGE_PATHS = [
    'C:/Users/breath03/Desktop/Smart_Closet/IMG_3260.jpg', 
    'C:/Users/breath03/Desktop/Smart_Closet/IMG_3261.jpg'
]

# 3. 결과 이미지를 저장할 폴더
OUTPUT_DIR = 'test_results'
# -----------------------------------------

def predict_my_clothes():
    """
    훈련된 YOLO 모델을 불러와 새로운 이미지들을 예측하고,
    좌표 값과 시각화된 결과 이미지를 저장합니다.
    """
    # 결과 저장 폴더 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 1. 훈련된 모델 불러오기
    try:
        model = YOLO(MODEL_PATH)
        print(f"'{MODEL_PATH}' 모델을 성공적으로 불러왔습니다.")
    except Exception as e:
        print(f"오류: 모델을 불러오는 데 실패했습니다. 경로를 확인해주세요. 에러: {e}")
        return

    # 2. 이미지들을 하나씩 모델에 넣어 예측 실행
    for img_path in IMAGE_PATHS:
        if not os.path.exists(img_path):
            print(f"경고: '{img_path}' 파일을 찾을 수 없습니다. 건너뜁니다.")
            continue

        print(f"\n--- '{img_path}' 이미지 분석 시작 ---")
        
        # 모델 예측
        results = model(img_path)

        # 3. 결과에서 좌표 값과 정보 추출하기
        for r in results:
            boxes = r.boxes
            if len(boxes) == 0:
                print("-> 탐지된 객체가 없습니다.")
                continue

            for box in boxes:
                # 클래스 이름 (상의, 하의 등)
                class_name = model.names[int(box.cls[0])]
                
                # 좌표 값 (x1, y1, x2, y2)
                coordinates = [int(x) for x in box.xyxy[0].tolist()]
                
                # 신뢰도 점수 (모델이 얼마나 확신하는지)
                confidence = float(box.conf[0])
                
                # 터미널에 결과 출력
                print(f"-> 탐지된 객체: {class_name}")
                print(f"   좌표 (x1, y1, x2, y2): {coordinates}")
                print(f"   신뢰도: {confidence:.2f}") # 소수점 2자리까지
        
        # 4. 시각화된 결과 이미지 저장하기
        # results 객체의 plot() 메소드는 바운딩 박스가 그려진 이미지를 반환합니다.
        result_image_array = r.plot() 
        result_image = Image.fromarray(result_image_array[..., ::-1]) # BGR -> RGB 변환
        
        output_path = os.path.join(OUTPUT_DIR, os.path.basename(img_path))
        result_image.save(output_path)
        print(f"-> 분석 결과 이미지를 '{output_path}'에 저장했습니다.")


if __name__ == '__main__':
    predict_my_clothes()

'C:/Users/breath03/runs/detect/train5/weights/best.pt' 모델을 성공적으로 불러왔습니다.

--- 'C:/Users/breath03/Desktop/Smart_Closet/IMG_3260.jpg' 이미지 분석 시작 ---

image 1/1 C:\Users\breath03\Desktop\Smart_Closet\IMG_3260.jpg: 640x480 1 , 66.0ms
Speed: 3.0ms preprocess, 66.0ms inference, 80.0ms postprocess per image at shape (1, 3, 640, 480)
-> 탐지된 객체: 원피스
   좌표 (x1, y1, x2, y2): [51, 174, 2866, 3788]
   신뢰도: 0.69
-> 분석 결과 이미지를 'test_results\IMG_3260.jpg'에 저장했습니다.

--- 'C:/Users/breath03/Desktop/Smart_Closet/IMG_3261.jpg' 이미지 분석 시작 ---

image 1/1 C:\Users\breath03\Desktop\Smart_Closet\IMG_3261.jpg: 640x480 1 , 12.0ms
Speed: 5.0ms preprocess, 12.0ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 480)
-> 탐지된 객체: 하의
   좌표 (x1, y1, x2, y2): [295, 142, 2897, 3935]
   신뢰도: 0.86
-> 분석 결과 이미지를 'test_results\IMG_3261.jpg'에 저장했습니다.


In [1]:
# 룩 이미지에서 개별 옷 이미지를 잘라내고 새로운 dataset을 생성하는 최종 스크립트

import pandas as pd
import os
from PIL import Image

# --- 설정 (백지민님이 확인할 부분) ---
# 1. 원본 데이터 경로
csv_path = './labeled_fashion_data(polygon_features).csv'
image_root_folder = './Training/원천데이터'

# 2. 새로 생성될 데이터셋을 저장할 경로
output_image_folder = './cropped_images'
new_metadata_csv_path = './cropped_images_metadata.csv'
# -----------------------------------------

def create_dataset_final():
    """
    '이미지 식별자'를 키로 사용하여 하위 폴더까지 탐색해 이미지를 찾고,
    데이터셋을 생성하는 최종 스크립트입니다.
    """
    # 결과물 저장 폴더 생성
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)

    # CSV 파일 읽기
    print(f"'{csv_path}' 파일을 읽는 중입니다...")
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"오류: '{csv_path}' 파일을 찾을 수 없습니다. 스크립트와 CSV 파일이 같은 위치에 있는지 확인해주세요.")
        return

    # 이미지 경로 맵 생성 (매우 중요!)
    # '원천데이터' 폴더와 그 안의 모든 하위 폴더를 탐색하여
    # '이미지 식별자'를 키로, 실제 파일 경로를 값으로 하는 딕셔너리를 만듭니다.
    print(f"'{image_root_folder}'와 모든 하위 폴더에서 이미지 파일을 찾는 중입니다...")
    if not os.path.isdir(image_root_folder):
        print(f"오류: '{image_root_folder}' 디렉토리를 찾을 수 없습니다.")
        return
        
    image_path_map = {}
    for root, _, files in os.walk(image_root_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                # 파일 이름에서 확장자를 제거한 부분을 키로 사용 (예: '994962')
                identifier = os.path.splitext(file)[0]
                image_path_map[identifier] = os.path.join(root, file)
    print(f"총 {len(image_path_map)}개의 이미지 경로를 찾았습니다.")

    # 새 메타데이터 리스트 초기화
    new_metadata_list = []
    total_rows = df.shape[0]
    print(f"총 {total_rows}개의 CSV 데이터 처리를 시작합니다...")

    # 데이터프레임 순회
    for index, row in df.iterrows():
        if (index + 1) % 1000 == 0:
            print(f"진행 상황: {index + 1} / {total_rows}")

        # '이미지 식별자'를 문자열로 변환하여 키로 사용
        identifier_key = str(row['이미지 식별자'])

        # 맵에서 해당 식별자를 가진 이미지 경로를 찾음
        if identifier_key not in image_path_map:
            continue # 이미지를 찾지 못하면 건너뛰기

        original_image_path = image_path_map[identifier_key]
        try:
            original_image = Image.open(original_image_path)
        except Exception:
            continue

        style = row['스타일_스타일']
        # 새 파일 이름에 '이미지 식별자'를 사용하여 더 명확하게 만듦
        base_name = identifier_key 

        item_types = {'상의': 'top', '하의': 'bottom', '아우터': 'outer', '원피스': 'onepiece'}

        for kor_type, eng_type in item_types.items():
            x_col, y_col, w_col, h_col = f'{kor_type}_렉트_X좌표', f'{kor_type}_렉트_Y좌표', f'{kor_type}_렉트_가로', f'{kor_type}_렉트_세로'
            cat_col = f'{kor_type}_라벨_카테고리'

            if x_col in row and row.get(x_col, 0) > 0 and pd.notna(row.get(cat_col)):
                x, y, w, h = row[x_col], row[y_col], row[w_col], row[h_col]
                category = row[cat_col]
                bbox = (x, y, x + w, y + h)
                
                width, height = original_image.size
                if not (0 <= bbox[0] < bbox[2] <= width and 0 <= bbox[1] < bbox[3] <= height):
                    continue

                try:
                    cropped_image = original_image.crop(bbox)
                    # 파일 이름 형식: {이미지식별자}_{옷종류}_{CSV행번호}.jpg
                    new_image_name = f"{base_name}_{eng_type}_{index}.jpg"
                    cropped_image_path = os.path.join(output_image_folder, new_image_name)
                    cropped_image.save(cropped_image_path)
                    new_metadata_list.append({
                        'image_name': new_image_name, 'category': category,
                        'style': style, 'original_image_identifier': base_name
                    })
                except Exception:
                    continue

    # 최종 결과 저장 및 출력
    if not new_metadata_list:
        print("\n" + "="*50)
        print("⚠️ 처리된 이미지가 없습니다. 다음을 확인해주세요:")
        print("1. CSV의 '이미지 식별자'와 실제 이미지 파일 이름(확장자 제외)이 일치하는지")
        print("2. 'Training/원천데이터' 폴더 안에 룩 스타일별 하위 폴더와 이미지들이 있는지")
        print("="*50)
        return

    new_df = pd.DataFrame(new_metadata_list)
    new_df.to_csv(new_metadata_csv_path, index=False, encoding='utf-8-sig')

    print("\n" + "="*50)
    print("🎉 데이터셋 생성이 성공적으로 완료되었습니다! 🎉")
    print(f"🖼️  잘라낸 이미지들은 '{output_image_folder}' 폴더에 저장되었습니다.")
    print(f"📊 새로운 메타데이터는 '{new_metadata_csv_path}' 파일에 저장되었습니다.")
    print(f"총 {len(new_metadata_list)}개의 새로운 개별 옷 데이터가 생성되었습니다.")
    print("="*50)

if __name__ == '__main__':
    create_dataset_final()

'./labeled_fashion_data(polygon_features).csv' 파일을 읽는 중입니다...


C:\Users\user\AppData\Local\Temp\ipykernel_18456\3795998490.py:29: DtypeWarning: Columns (26,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


'./Training/원천데이터'와 모든 하위 폴더에서 이미지 파일을 찾는 중입니다...
총 643265개의 이미지 경로를 찾았습니다.
총 41240개의 CSV 데이터 처리를 시작합니다...
진행 상황: 1000 / 41240
진행 상황: 2000 / 41240
진행 상황: 3000 / 41240
진행 상황: 4000 / 41240
진행 상황: 5000 / 41240
진행 상황: 6000 / 41240
진행 상황: 7000 / 41240
진행 상황: 8000 / 41240
진행 상황: 9000 / 41240
진행 상황: 10000 / 41240
진행 상황: 11000 / 41240
진행 상황: 12000 / 41240
진행 상황: 13000 / 41240
진행 상황: 14000 / 41240
진행 상황: 15000 / 41240
진행 상황: 16000 / 41240
진행 상황: 17000 / 41240
진행 상황: 18000 / 41240
진행 상황: 19000 / 41240
진행 상황: 20000 / 41240
진행 상황: 21000 / 41240
진행 상황: 22000 / 41240
진행 상황: 23000 / 41240
진행 상황: 24000 / 41240
진행 상황: 25000 / 41240
진행 상황: 26000 / 41240
진행 상황: 27000 / 41240
진행 상황: 28000 / 41240
진행 상황: 29000 / 41240
진행 상황: 30000 / 41240
진행 상황: 31000 / 41240
진행 상황: 32000 / 41240
진행 상황: 33000 / 41240
진행 상황: 34000 / 41240
진행 상황: 35000 / 41240
진행 상황: 36000 / 41240
진행 상황: 37000 / 41240
진행 상황: 38000 / 41240
진행 상황: 39000 / 41240
진행 상황: 40000 / 41240
진행 상황: 41000 / 41240

🎉 데이터셋 생성이 성공적으로 완료되었습니다! 🎉
🖼️  잘라낸 이미지들

In [1]:
# ViT 모델을 finetuning하는 코드

import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image
from transformers import (
    AutoImageProcessor,
    AutoModel,
    TrainingArguments,
    Trainer
)
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score
import os

# --- 설정 ---
CSV_PATH = 'cropped_images_metadata.csv'
IMAGE_DIR = 'cropped_images'
MODEL_NAME = "google/vit-base-patch16-224-in21k"
OUTPUT_DIR = "fashion_classifier_model"
# ------------

# 1. 데이터셋 클래스 정의
class FashionDataset(Dataset):
    def __init__(self, csv_path, image_dir, processor):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.processor = processor
        
        # 카테고리와 스타일 라벨을 숫자로 변환 (Label Encoding)
        self.category_labels = self.df['category'].astype('category').cat.codes
        self.style_labels = self.df['style'].astype('category').cat.codes
        
        # 나중에 숫자를 다시 원래 이름으로 바꾸기 위해 매핑 정보 저장
        self.id2cat = dict(enumerate(self.df['category'].astype('category').cat.categories))
        self.id2style = dict(enumerate(self.df['style'].astype('category').cat.categories))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row['image_name'])
        image = Image.open(image_path).convert("RGB")
        
        # 이미지를 모델이 이해할 수 있는 형태로 변환
        inputs = self.processor(images=image, return_tensors="pt")
        
        return {
            "pixel_values": inputs.pixel_values.squeeze(),
            "category_labels": torch.tensor(self.category_labels[idx], dtype=torch.long),
            "style_labels": torch.tensor(self.style_labels[idx], dtype=torch.long)
        }

# 2. Multi-Task 모델 설계
class MultiTaskClassifier(nn.Module):
    def __init__(self, num_categories, num_styles):
        super().__init__()
        self.body = AutoModel.from_pretrained(MODEL_NAME)
        hidden_size = self.body.config.hidden_size
        self.category_head = nn.Linear(hidden_size, num_categories)
        self.style_head = nn.Linear(hidden_size, num_styles)

    def forward(self, pixel_values, category_labels=None, style_labels=None):
        features = self.body(pixel_values=pixel_values).last_hidden_state[:, 0, :]
        
        category_logits = self.category_head(features)
        style_logits = self.style_head(features)

        # 훈련 시에는 두 개의 loss를 계산하여 합침
        if category_labels is not None and style_labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss_category = loss_fct(category_logits, category_labels)
            loss_style = loss_fct(style_logits, style_labels)
            total_loss = loss_category + loss_style
            return {"loss": total_loss, "category_logits": category_logits, "style_logits": style_logits}
        
        return {"category_logits": category_logits, "style_logits": style_logits}

# 3. Multi-Task Trainer 정의 (두 개의 출력을 평가하기 위함)
class MultiTaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        return (outputs['loss'], outputs) if return_outputs else outputs['loss']

def compute_metrics(eval_pred):
    category_logits, style_logits = eval_pred.predictions
    category_labels, style_labels = eval_pred.label_ids

    cat_preds = np.argmax(category_logits, axis=1)
    style_preds = np.argmax(style_logits, axis=1)

    return {
        'category_accuracy': accuracy_score(category_labels, cat_preds),
        'style_accuracy': accuracy_score(style_labels, style_preds)
    }

# --- 메인 실행 ---
if __name__ == '__main__':
    # 데이터셋 준비
    processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
    full_dataset = FashionDataset(CSV_PATH, IMAGE_DIR, processor)
    
    # 데이터셋을 훈련용과 테스트용으로 분리 (9:1)
    train_size = int(0.9 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

    # 모델 생성
    num_categories = len(full_dataset.id2cat)
    num_styles = len(full_dataset.id2style)
    model = MultiTaskClassifier(num_categories, num_styles)

    # 훈련 규칙 설정
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=16,
        eval_strategy="epoch",
        num_train_epochs=5, # 에포크는 필요에 따라 조절
        save_strategy="epoch",
        logging_steps=100,
        load_best_model_at_end=True,
    )

    # 훈련 관리자 생성
    trainer = MultiTaskTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )

    # 훈련 시작!
    trainer.train()

    # 훈련된 모델과 라벨 정보 저장
    trainer.save_model(OUTPUT_DIR)
    torch.save({
        'id2cat': full_dataset.id2cat,
        'id2style': full_dataset.id2style
    }, os.path.join(OUTPUT_DIR, 'label_maps.pth'))

    print("🎉 훈련이 완료되었습니다! 모델이 '{OUTPUT_DIR}' 폴더에 저장되었습니다.")

c:\Users\breath03\Desktop\Smart_Closet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Epoch,Training Loss,Validation Loss,Category Accuracy,Style Accuracy
1,0.975800,0.945912,0.863751,0.856199
2,0.392500,0.538363,0.921020,0.915041
3,0.158300,0.466980,0.942417,0.929515
4,0.050300,0.408854,0.948395,0.941787
5,0.012300,0.396294,0.952486,0.947451


🎉 훈련이 완료되었습니다! 모델이 '{OUTPUT_DIR}' 폴더에 저장되었습니다.


In [4]:
# print(train_df.shape)
# print(train_df.head())
# print(train_df.isnull().sum())
# print(train_yolo_df.head())
import torch
# print(torch.cuda.is_available())
print(torch.backends.cudnn.is_available())
print(torch.backends.cudnn.version())

True
90100
